<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_ShuffleNet/test_sample_ShuffleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

In [ ]:
! pip install datasets
! pip install wget

In [1]:
# ! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/

/content/PyTorch-Architectures


In [2]:
from tqdm.auto import tqdm
import torch
from toolkit.custom_dataset_cv import DataLoaderCIFAR10Classification
from toolkit.metrics import cv_compute_accuracy
from toolkit.utils import get_optimal_batchsize, dict_to_device
from modeling_ShuffleNet.model import ShuffleNet
from modeling_ShuffleNet.config import ShuffleNetConfig

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
config = ShuffleNetConfig()
model = ShuffleNet(config)
model.to(device)

In [4]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Trainable Parameters: ', params)

Trainable Parameters:  960706


In [5]:
train_loader = DataLoaderCIFAR10Classification(resize=224, train=True)
valid_loader = DataLoaderCIFAR10Classification(resize=224, train=False)

cifar10 exists...
cifar10 exists...


In [8]:
# get_optimal_batchsize(train_loader.dataset, model) --> 256

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


256

In [6]:
# Hyperparameters
BS = 256
EPOCHS = 3
LR = 5e-3

In [8]:
train_loader = train_loader.return_dataloader(batch_size=BS, shuffle=True)
valid_loader = valid_loader.return_dataloader(batch_size=BS, shuffle=False)
print('Length of Train Loader: ', len(train_loader))
print('Length of Valid Loader: ', len(valid_loader))

Length of Train Loader:  196
Length of Valid Loader:  40


In [9]:
# Sanity check forward pass
model.eval()
with torch.set_grad_enabled(False):
  for sample in train_loader:
    outputs = model(**dict_to_device(sample, device))
    loss, logits = outputs[0], outputs[1]
    print(logits.shape, loss.item())
    break

torch.Size([256, 10]) 2130.892333984375


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [11]:
num_training_steps = len(train_loader) * EPOCHS
progress_bar = tqdm(range(num_training_steps))

for epoch in range(EPOCHS):
  model.train()
  for sample in train_loader:
      outputs = model(**dict_to_device(sample, device))
      loss = outputs[0]
      loss.backward()

      optimizer.step()
      optimizer.zero_grad()
      progress_bar.update(1)
  model.eval()
  with torch.set_grad_enabled(False):
    train_acc = cv_compute_accuracy(model, train_loader, device)
    valid_acc = cv_compute_accuracy(model, valid_loader, device)
  print('Train Accuracy: %.2f%% || Valid Accuracy: %.2f%%' % (train_acc, valid_acc))

Train Accuracy: 58.26% || Valid Accuracy: 56.25%
Train Accuracy: 65.44% || Valid Accuracy: 62.91%
Train Accuracy: 77.77% || Valid Accuracy: 73.23%
